In [3]:
import pandas as pd
import json
from collections import defaultdict

In [5]:
# Read the GeoJSON file
with open('LRTMRT.geojson', 'r', encoding='utf-8') as f:
    geojson_data = json.load(f)

# Dictionary to store station data: {station_name: {exit_code: (lon, lat)}}
# Using dict to automatically handle duplicates (keep first occurrence)
station_data = defaultdict(dict)

# Extract data from features
for feature in geojson_data['features']:
    station_name = feature['properties']['STATION_NA']
    exit_code = feature['properties']['EXIT_CODE']
    coordinates = feature['geometry']['coordinates']  # [longitude, latitude]
    lon, lat = coordinates[0], coordinates[1]
    
    # Only add if exit_code doesn't already exist for this station
    if exit_code not in station_data[station_name]:
        station_data[station_name][exit_code] = (lon, lat)

# Convert to DataFrame
rows = []
for station_name, exits_dict in station_data.items():
    # Sort exits by exit code (Exit A, Exit B, etc.)
    exits_sorted = sorted(exits_dict.items(), key=lambda x: x[0])
    
    row = {'Station Name': station_name}
    
    # Add columns for each exit
    for idx, (exit_code, (lon, lat)) in enumerate(exits_sorted, 1):
        row[f'Exit {idx} Longitude'] = lon
        row[f'Exit {idx} Latitude'] = lat
    
    rows.append(row)

# Create DataFrame
df = pd.DataFrame(rows)

# Sort by station name
df = df.sort_values('Station Name').reset_index(drop=True)

# Display first few rows
print(f"Total stations: {len(df)}")
print(f"\nColumns: {list(df.columns)}")
df.head(10)


Total stations: 186

Columns: ['Station Name', 'Exit 1 Longitude', 'Exit 1 Latitude', 'Exit 2 Longitude', 'Exit 2 Latitude', 'Exit 3 Longitude', 'Exit 3 Latitude', 'Exit 4 Longitude', 'Exit 4 Latitude', 'Exit 5 Longitude', 'Exit 5 Latitude', 'Exit 6 Longitude', 'Exit 6 Latitude', 'Exit 7 Longitude', 'Exit 7 Latitude', 'Exit 8 Longitude', 'Exit 8 Latitude', 'Exit 9 Longitude', 'Exit 9 Latitude', 'Exit 10 Longitude', 'Exit 10 Latitude', 'Exit 11 Longitude', 'Exit 11 Latitude', 'Exit 12 Longitude', 'Exit 12 Latitude', 'Exit 13 Longitude', 'Exit 13 Latitude']


,Station Name,Exit 1 Longitude,Exit 1 Latitude,Exit 2 Longitude,Exit 2 Latitude,Exit 3 Longitude,Exit 3 Latitude,Exit 4 Longitude,Exit 4 Latitude,Exit 5 Longitude,...,Exit 9 Longitude,Exit 9 Latitude,Exit 10 Longitude,Exit 10 Latitude,Exit 11 Longitude,Exit 11 Latitude,Exit 12 Longitude,Exit 12 Latitude,Exit 13 Longitude,Exit 13 Latitude
0,ADMIRALTY MRT STATION,103.800990,1.440346,103.800533,1.440152,103.800542,1.440596,103.801365,1.440533,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ALJUNIED MRT STATION,103.882497,1.316238,103.882473,1.316488,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ANG MO KIO MRT STATION,103.849939,1.369465,103.849596,1.369685,103.849343,1.369441,103.849825,1.369882,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,BAKAU LRT STATION,103.905446,1.388185,103.905482,1.387841,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,BANGKIT LRT STATION,103.772591,1.379853,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,BARTLEY MRT STATION,103.880256,1.342249,103.878909,1.343501,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,BAYFRONT MRT STATION,103.858732,1.280967,103.859759,1.282705,103.859597,1.282835,103.859494,1.282834,103.858441,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,BAYSHORE MRT STATION,103.941262,1.311899,103.940673,1.312184,103.940721,1.313568,103.942676,1.313526,103.942584,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,BEAUTY WORLD MRT STATION,103.775984,1.341696,103.775451,1.341582,103.775762,1.339982,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,BEDOK MRT STATION,103.929157,1.323569,103.929673,1.324268,103.929404,1.324075,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# Save to CSV
df.to_csv('mrtlrt_stations.csv', index=False)
print("CSV file saved as 'mrtlrt_stations.csv'")


CSV file saved as 'mrtlrt_stations.csv'


In [7]:
# Alternative: Create a simplified version with one coordinate per station
# Using centroid (geometric center) of all exits - better than simple average for geographic coordinates

rows_simple = []
for station_name, exits_dict in station_data.items():
    # Get all coordinates for this station
    coords = list(exits_dict.values())  # [(lon, lat), ...]
    
    # Calculate centroid (geometric center)
    # For geographic coordinates, simple average works well for small areas like Singapore
    avg_lon = sum(lon for lon, lat in coords) / len(coords)
    avg_lat = sum(lat for lon, lat in coords) / len(coords)
    
    rows_simple.append({
        'Station Name': station_name,
        'Longitude': avg_lon,
        'Latitude': avg_lat,
        'Number of Exits': len(coords)
    })

# Create simplified DataFrame
df_simple = pd.DataFrame(rows_simple)
df_simple = df_simple.sort_values('Station Name').reset_index(drop=True)

print(f"Simplified version with centroid coordinates:")
print(f"Total stations: {len(df_simple)}")
df_simple.head(10)


Simplified version with centroid coordinates:
Total stations: 186


,Station Name,Longitude,Latitude,Number of Exits
0,ADMIRALTY MRT STATION,103.800857,1.440407,4
1,ALJUNIED MRT STATION,103.882485,1.316363,2
2,ANG MO KIO MRT STATION,103.849676,1.369618,4
3,BAKAU LRT STATION,103.905464,1.388013,2
4,BANGKIT LRT STATION,103.772591,1.379853,1
5,BARTLEY MRT STATION,103.879582,1.342875,2
6,BAYFRONT MRT STATION,103.859205,1.282089,5
7,BAYSHORE MRT STATION,103.941583,1.312815,5
8,BEAUTY WORLD MRT STATION,103.775732,1.341087,3
9,BEDOK MRT STATION,103.929412,1.323971,3


In [8]:
# Save simplified version to CSV
df_simple.to_csv('mrtlrt_stations_simple.csv', index=False)
print("Simplified CSV file saved as 'mrtlrt_stations_simple.csv'")
print("\nNote: This uses the centroid (geometric center) of all exits for each station.")
print("This is better than a simple average for geographic coordinates.")


Simplified CSV file saved as 'mrtlrt_stations_simple.csv'

Note: This uses the centroid (geometric center) of all exits for each station.
This is better than a simple average for geographic coordinates.


In [ ]:
manual_opening_dates = {
    # North-South Line stations
    'ADMIRALTY MRT STATION': '10 February 1996',
    'SEMBAWANG MRT STATION': '10 February 1996',
    'CANBERRA MRT STATION': '2 November 2019',
    'YISHUN MRT STATION': '20 December 1988',
    'KHATIB MRT STATION': '20 December 1988',
    'YIO CHU KANG MRT STATION': '7 November 1987',
    'ANG MO KIO MRT STATION': '7 November 1987',
    'BRADDELL MRT STATION': '7 November 1987',
    'BISHAN MRT STATION': '7 November 1987',
    'TOA PAYOH MRT STATION': '7 November 1987',
    'NOVENA MRT STATION': '12 December 1987',
    'NEWTON MRT STATION': '12 December 1987',
    'ORCHARD MRT STATION': '12 December 1987',
    'SOMERSET MRT STATION': '12 December 1987',
    'DHOBY GHAUT MRT STATION': '12 December 1987',
    'CITY HALL MRT STATION': '12 December 1987',
    'RAFFLES PLACE MRT STATION': '12 December 1987',
    'MARINA BAY MRT STATION': '4 November 1989',
    
    # East-West Line stations
    'PASIR RIS MRT STATION': '16 December 1989',
    'TAMPINES MRT STATION': '16 December 1989',
    'SIMEI MRT STATION': '16 December 1989',
    'TANAH MERAH MRT STATION': '4 November 1989',
    'BEDOK MRT STATION': '4 November 1989',
    'KEMBANGAN MRT STATION': '4 November 1989',
    'EUNOS MRT STATION': '4 November 1989',
    'PAYA LEBAR MRT STATION': '4 November 1989',
    'ALJUNIED MRT STATION': '4 November 1989',
    'KALLANG MRT STATION': '4 November 1989',
    'LAVENDER MRT STATION': '4 November 1989',
    'BUGIS MRT STATION': '4 November 1989',
    'TANJONG PAGAR MRT STATION': '12 December 1987',
    'OUTRAM PARK MRT STATION': '12 December 1987',
    'TIONG BAHRU MRT STATION': '12 March 1988',
    'REDHILL MRT STATION': '12 March 1988',
    'QUEENSTOWN MRT STATION': '12 March 1988',
    'COMMONWEALTH MRT STATION': '12 March 1988',
    'BUONA VISTA MRT STATION': '12 March 1988',
    'DOVER MRT STATION': '18 October 2001',
    'CLEMENTI MRT STATION': '12 March 1988',
    'JURONG EAST MRT STATION': '5 November 1988',
    'CHINESE GARDEN MRT STATION': '5 November 1988',
    'LAKESIDE MRT STATION': '5 November 1988',
    'BOON LAY MRT STATION': '6 July 1990',
    'PIONEER MRT STATION': '28 February 2009',
    'JOO KOON MRT STATION': '28 February 2009',
    'GUL CIRCLE MRT STATION': '18 June 2017',
    'TUAS CRESCENT MRT STATION': '18 June 2017',
    'TUAS WEST ROAD MRT STATION': '18 June 2017',
    'TUAS LINK MRT STATION': '18 June 2017',
    'EXPO MRT STATION': '10 January 2001',
    'CHANGI AIRPORT MRT STATION': '8 February 2002',
    
    # North East Line stations
    'HARBOURFRONT MRT STATION': '20 June 2003',
    'OUTRAM PARK MRT STATION': '20 June 2003',
    'CHINATOWN MRT STATION': '20 June 2003',
    'CLARKE QUAY MRT STATION': '20 June 2003',
    'DHOBY GHAUT MRT STATION': '20 June 2003',
    'LITTLE INDIA MRT STATION': '20 June 2003',
    'FARRER PARK MRT STATION': '20 June 2003',
    'BOON KENG MRT STATION': '20 June 2003',
    'POTONG PASIR MRT STATION': '20 June 2003',
    'WOODLEIGH MRT STATION': '15 January 2006',
    'SERANGOON MRT STATION': '20 June 2003',
    'KOVAN MRT STATION': '20 June 2003',
    'HOUGANG MRT STATION': '20 June 2003',
    'BUANGKOK MRT STATION': '15 January 2006',
    'SENGKANG MRT STATION': '20 June 2003',
    'PUNGGOL MRT STATION': '20 June 2003',
    
    # Circle Line stations
    'BRAS BASAH MRT STATION': '17 April 2010',
    'ESPLANADE MRT STATION': '17 April 2010',
    'PROMENADE MRT STATION': '17 April 2010',
    'NICOLL HIGHWAY MRT STATION': '17 April 2010',
    'STADIUM MRT STATION': '17 April 2010',
    'MOUNTBATTEN MRT STATION': '17 April 2010',
    'DAKOTA MRT STATION': '17 April 2010',
    'MACPHERSON MRT STATION': '17 April 2010',
    'TAI SENG MRT STATION': '17 April 2010',
    'BARTLEY MRT STATION': '17 April 2010',
    'LORONG CHUAN MRT STATION': '28 May 2009',
    'MARYMOUNT MRT STATION': '28 May 2009',
    'CALDECOTT MRT STATION': '8 October 2011',
    'BOTANIC GARDENS MRT STATION': '8 October 2011',
    'FARRER ROAD MRT STATION': '8 October 2011',
    'HOLLAND VILLAGE MRT STATION': '8 October 2011',
    'ONE-NORTH MRT STATION': '8 October 2011',
    'KENT RIDGE MRT STATION': '8 October 2011',
    'HAW PAR VILLA MRT STATION': '8 October 2011',
    'PASIR PANJANG MRT STATION': '8 October 2011',
    'LABRADOR PARK MRT STATION': '8 October 2011',
    'TELOK BLANGAH MRT STATION': '8 October 2011',
    'BAYFRONT MRT STATION': '14 January 2012',
    
    # Downtown Line stations
    'BUKIT PANJANG MRT STATION': '27 December 2015',
    'CASHEW MRT STATION': '27 December 2015',
    'HILLVIEW MRT STATION': '27 December 2015',
    'BEAUTY WORLD MRT STATION': '27 December 2015',
    'KING ALBERT PARK MRT STATION': '27 December 2015',
    'SIXTH AVENUE MRT STATION': '27 December 2015',
    'TAN KAH KEE MRT STATION': '27 December 2015',
    'STEVENS MRT STATION': '27 December 2015',
    'ROCHOR MRT STATION': '27 December 2015',
    'DOWNTOWN MRT STATION': '22 December 2013',
    'TELOK AYER MRT STATION': '22 December 2013',
    'FORT CANNING MRT STATION': '21 October 2017',
    'BENCOOLEN MRT STATION': '21 October 2017',
    'JALAN BESAR MRT STATION': '21 October 2017',
    'BENDEMEER MRT STATION': '21 October 2017',
    'GEYLANG BAHRU MRT STATION': '21 October 2017',
    'MATTAR MRT STATION': '21 October 2017',
    'UBI MRT STATION': '21 October 2017',
    'KAKI BUKIT MRT STATION': '21 October 2017',
    'BEDOK RESERVOIR MRT STATION': '21 October 2017',
    'TAMPINES WEST MRT STATION': '21 October 2017',
    'TAMPINES EAST MRT STATION': '21 October 2017',
    'UPPER CHANGI MRT STATION': '21 October 2017',
    
    # Thomson-East Coast Line stations
    'WOODLANDS NORTH MRT STATION': '31 January 2020',
    'WOODLANDS SOUTH MRT STATION': '31 January 2020',
    'LENTOR MRT STATION': '13 November 2022',
    'MAYFLOWER MRT STATION': '13 November 2022',
    'BRIGHT HILL MRT STATION': '13 November 2022',
    'UPPER THOMSON MRT STATION': '13 November 2022',
    'MOUNT PLEASANT MRT STATION': '13 November 2022',
    'NAPIER MRT STATION': '13 November 2022',
    'ORCHARD BOULEVARD MRT STATION': '13 November 2022',
    'GREAT WORLD MRT STATION': '13 November 2022',
    'HAVELOCK MRT STATION': '13 November 2022',
    'MAXWELL MRT STATION': '13 November 2022',
    'SHELTON MRT STATION': '13 November 2022',
    'MARINA SOUTH MRT STATION': '13 November 2022',
    'GARDENS BY THE BAY MRT STATION': '13 November 2022',
    'TANJONG RHU MRT STATION': '23 June 2024',
    'KATONG PARK MRT STATION': '23 June 2024',
    'TANJONG KATONG MRT STATION': '23 June 2024',
    'MARINE PARADE MRT STATION': '23 June 2024',
    'MARINE TERRACE MRT STATION': '23 June 2024',
    'SIGLAP MRT STATION': '23 June 2024',
    'BAYSHORE MRT STATION': '23 June 2024',
    'BEDOK SOUTH MRT STATION': '23 June 2024',
    'SUNRISE WAY MRT STATION': '23 June 2024',
    'UPPER EAST COAST MRT STATION': '23 June 2024',
    
    # LRT Stations (Bukit Panjang LRT)
    'CHOA CHU KANG LRT STATION': '6 November 1999',
    'SOUTH VIEW LRT STATION': '6 November 1999',
    'KEAT HONG LRT STATION': '6 November 1999',
    'TECK WHYE LRT STATION': '6 November 1999',
    'PHOENIX LRT STATION': '6 November 1999',
    'BUKIT PANJANG LRT STATION': '6 November 1999',
    'PETIR LRT STATION': '6 November 1999',
    'PENDING LRT STATION': '6 November 1999',
    'BANGKIT LRT STATION': '6 November 1999',
    'FAJAR LRT STATION': '6 November 1999',
    'SEGAR LRT STATION': '6 November 1999',
    'JELAPANG LRT STATION': '6 November 1999',
    'SENJA LRT STATION': '6 November 1999',
    'TEN MILE JUNCTION LRT STATION': '6 November 1999',
    
    # LRT Stations (Sengkang LRT)
    'SENGKANG LRT STATION': '18 January 2003',
    'COMPASSVALE LRT STATION': '18 January 2003',
    'RANGGUNG LRT STATION': '18 January 2003',
    'RUMBIA LRT STATION': '18 January 2003',
    'BAKAU LRT STATION': '18 January 2003',
    'KANGKAR LRT STATION': '18 January 2003',
    'RENJONG LRT STATION': '18 January 2003',
    
    # LRT Stations (Punggol LRT)
    'COVE LRT STATION': '29 January 2005',
    'MERIDIAN LRT STATION': '29 January 2005',
    'CORAL EDGE LRT STATION': '29 January 2005',
    'RIVIERA LRT STATION': '29 January 2005',
    'KADAS LRT STATION': '29 January 2005',
    'PUNGGOL POINT LRT STATION': '29 January 2005',
    'SAM KEE LRT STATION': '29 January 2005',
    'TEKNOLOGI LRT STATION': '29 January 2005',
    'DAMAI LRT STATION': '29 January 2005',
    'SOO TECK LRT STATION': '29 January 2005',
    'SAMUDERA LRT STATION': '29 January 2005',
    'NIBONG LRT STATION': '29 January 2005',
    'SUMANG LRT STATION': '29 January 2005',
    'TECK LEE LRT STATION': '29 January 2005',
    'KADALOOR LRT STATION': '29 January 2005',
    'OASIS LRT STATION': '29 January 2005',
    'THANGGAM LRT STATION': '29 January 2005',
    'TONGKANG LRT STATION': '29 January 2005',
    
    # Additional stations from CSV
    'BEDOK NORTH MRT STATION': '17 November 2017',  
    'LAVENDER MRT STATION': '4 November 1989',  
    'MARINA SOUTH PIER MRT STATION': '23 November 2014',
    'SHENTON WAY MRT STATION': '23 November 2014',
    'SPRINGLEAF MRT STATION': '31 January 2020',
    'BUKIT BATOK MRT STATION': '10 March 1990',
    'BUKIT GOMBAK MRT STATION': '10 March 1990',
    'CHENG LIM LRT STATION': '18 January 2003',
    'FARMWAY LRT STATION': '18 January 2003',
    'FERNVALE LRT STATION': '18 January 2003',
    'KUPANG LRT STATION': '18 January 2003',
    'LAYAR LRT STATION': '18 January 2003',
    'SHELTON MRT STATION': '13 November 2022',  
    'CHOA CHU KANG MRT STATION': '10 March 1990',  
    'KRANJI MRT STATION': '10 February 1996',
    'MARSILING MRT STATION': '10 February 1996',
    'PUNGGOL LRT STATION': '29 January 2005',
    'WOODLANDS MRT STATION': '10 February 1996',
    'YEW TEE MRT STATION': '10 February 1996',
}

# Apply the manual mapping to df_simple
df_simple['Opening Date'] = df_simple['Station Name'].map(manual_opening_dates)

# Convert opening date to "YYYY-MM" format for comparison with housing dataset
def convert_to_year_month(date_str):
    """Convert 'DD Month YYYY' format to 'YYYY-MM' format"""
    if pd.isna(date_str) or not date_str:
        return None
    
    # Month name to number mapping
    month_map = {
        'January': '01', 'February': '02', 'March': '03', 'April': '04',
        'May': '05', 'June': '06', 'July': '07', 'August': '08',
        'September': '09', 'October': '10', 'November': '11', 'December': '12'
    }
    
    # Parse "DD Month YYYY" format
    parts = date_str.split()
    if len(parts) == 3:
        day, month, year = parts
        if month in month_map:
            return f"{year}-{month_map[month]}"
    
    return None

# Add the year-month column
df_simple['Opening Year-Month'] = df_simple['Opening Date'].apply(convert_to_year_month)

# Show results
print(f"Manual mapping contains {len(manual_opening_dates)} station entries")
print(f"\nFound opening dates for {df_simple['Opening Date'].notna().sum()} out of {len(df_simple)} stations")
print(f"Success rate: {df_simple['Opening Date'].notna().sum() / len(df_simple) * 100:.1f}%")

# Show stations with dates
print("\nStations WITH opening dates (first 20):")
with_dates = df_simple[df_simple['Opening Date'].notna()][['Station Name', 'Opening Date', 'Opening Year-Month']].head(20)
print(with_dates.to_string(index=False))

# Show stations without dates
print("\n\nStations WITHOUT opening dates:")
missing = df_simple[df_simple['Opening Date'].isna()]['Station Name'].tolist()
print(f"Total: {len(missing)} stations")
if len(missing) > 0:
    for station in missing[:30]:
        print(f"  - {station}")
    if len(missing) > 30:
        print(f"  ... and {len(missing) - 30} more")


Manual mapping contains 192 station entries

Found opening dates for 183 out of 186 stations
Success rate: 98.4%

Stations WITH opening dates (first 20):
               Station Name     Opening Date Opening Year-Month
      ADMIRALTY MRT STATION 10 February 1996            1996-02
       ALJUNIED MRT STATION  4 November 1989            1989-11
     ANG MO KIO MRT STATION  7 November 1987            1987-11
          BAKAU LRT STATION  18 January 2003            2003-01
        BANGKIT LRT STATION  6 November 1999            1999-11
        BARTLEY MRT STATION    17 April 2010            2010-04
       BAYFRONT MRT STATION  14 January 2012            2012-01
       BAYSHORE MRT STATION     23 June 2024            2024-06
   BEAUTY WORLD MRT STATION 27 December 2015            2015-12
          BEDOK MRT STATION  4 November 1989            1989-11
    BEDOK NORTH MRT STATION 17 November 2017            2017-11
BEDOK RESERVOIR MRT STATION  21 October 2017            2017-10
      BENCOOLE

In [10]:
# Save the updated dataframe with opening dates to CSV
df_simple.to_csv('mrtlrt_stations_simple.csv', index=False)
print("Updated CSV file saved as 'mrtlrt_stations_simple.csv' with opening dates!")
print(f"\nFinal summary:")
print(f"  Total stations: {len(df_simple)}")
print(f"  Stations with opening dates: {df_simple['Opening Date'].notna().sum()}")
print(f"  Stations without opening dates: {df_simple['Opening Date'].isna().sum()}")
print(f"\nColumns in CSV:")
print(f"  {', '.join(df_simple.columns.tolist())}")
print(f"\nSample of date formats:")
sample = df_simple[df_simple['Opening Date'].notna()][['Station Name', 'Opening Date', 'Opening Year-Month']].head(5)
print(sample.to_string(index=False))


Updated CSV file saved as 'mrtlrt_stations_simple.csv' with opening dates!

Final summary:
  Total stations: 186
  Stations with opening dates: 183
  Stations without opening dates: 3

Columns in CSV:
  Station Name, Longitude, Latitude, Number of Exits, Opening Date, Opening Year-Month

Sample of date formats:
          Station Name     Opening Date Opening Year-Month
 ADMIRALTY MRT STATION 10 February 1996            1996-02
  ALJUNIED MRT STATION  4 November 1989            1989-11
ANG MO KIO MRT STATION  7 November 1987            1987-11
     BAKAU LRT STATION  18 January 2003            2003-01
   BANGKIT LRT STATION  6 November 1999            1999-11


In [22]:
!pip install pyproj

zsh:1: /Users/hemantdantam/Desktop/SUTD_class_files/sutdenv/bin/pip: bad interpreter: /Users/hemantdantam/Desktop/SUTD_files/sutdenv/bin/python: no such file or directory


In [14]:
# Convert WGS84 coordinates (Longitude, Latitude) to SVY21 coordinates (X, Y)
# SVY21 is Singapore's national coordinate system (EPSG:3414)
# WGS84 is the standard GPS coordinate system (EPSG:4326)

# Install pyproj if not available
try:
    from pyproj import Transformer
except ImportError:
    import subprocess
    import sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "pyproj"])
    from pyproj import Transformer

# Create transformer from WGS84 to SVY21
# WGS84: EPSG:4326 (longitude, latitude)
# SVY21: EPSG:3414 (X, Y in meters)
try:
    transformer = Transformer.from_crs("EPSG:4326", "EPSG:3414", always_xy=True)
    
    # Read the CSV file (or use df_simple if it's still in memory)
    # If df_simple is not available, read from CSV
    try:
        df_coords = df_simple.copy()
    except NameError:
        df_coords = pd.read_csv('mrtlrt_stations_simple.csv')
    
    # Convert coordinates for each row
    def convert_to_svy21(row):
        """Convert WGS84 lon/lat to SVY21 X/Y"""
        lon = row['Longitude']
        lat = row['Latitude']
        
        # Skip if coordinates are missing
        if pd.isna(lon) or pd.isna(lat):
            return pd.Series({'SVY21_X': None, 'SVY21_Y': None})
        
        # Transform coordinates
        x, y = transformer.transform(lon, lat)
        
        return pd.Series({'SVY21_X': x, 'SVY21_Y': y})
    
    # Apply conversion and add new columns
    svy21_coords = df_coords.apply(convert_to_svy21, axis=1)
    df_coords['SVY21_X'] = svy21_coords['SVY21_X']
    df_coords['SVY21_Y'] = svy21_coords['SVY21_Y']
    
    # Display results
    print("Coordinate conversion completed!")
    print(f"\nNew columns added: SVY21_X, SVY21_Y")
    print(f"\nSample of converted coordinates (first 5 stations):")
    sample = df_coords[['Station Name', 'Longitude', 'Latitude', 'SVY21_X', 'SVY21_Y']].head(5)
    print(sample.to_string(index=False))
    
    # Save updated CSV
    df_coords.to_csv('mrtlrt_stations_simple.csv', index=False)
    print(f"\n✓ Updated CSV saved with SVY21 coordinates!")
    print(f"\nFinal columns: {', '.join(df_coords.columns.tolist())}")
    
except Exception as e:
    print(f"Error during coordinate conversion: {e}")
    import traceback
    traceback.print_exc()


Coordinate conversion completed!

New columns added: SVY21_X, SVY21_Y

Sample of converted coordinates (first 5 stations):
          Station Name  Longitude  Latitude      SVY21_X      SVY21_Y
 ADMIRALTY MRT STATION 103.800857  1.440407 24387.582676 46898.423422
  ALJUNIED MRT STATION 103.882485  1.316363 33471.721150 33182.326750
ANG MO KIO MRT STATION 103.849676  1.369618 29820.353801 39070.947449
     BAKAU LRT STATION 103.905464  1.388013 36028.839300 41105.073650
   BANGKIT LRT STATION 103.772591  1.379853 21241.735400 40202.774300

✓ Updated CSV saved with SVY21 coordinates!

Final columns: Station Name, Longitude, Latitude, Number of Exits, Opening Date, Opening Year-Month, SVY21_X, SVY21_Y
